# 隐藏层

我们成功训练了第一个神经元网络模型，它可以根据天气预报来预测冰激凌的销量。

但冰激凌的销量真的是直接由天气情况决定吗？

事实上，天气并不是直接影响冰激凌销量的因素。天气首先影响的是人们的行为和偏好，例如是否愿意出门，以及是否想吃冰激凌：

* 天气凉爽时：人们更愿意出门，但未必想吃冰激凌；
* 天气酷热时：人们更想吃冰激凌，但可能不愿意出门；
* 天气寒冷时：人们既不愿意出门，也不想吃冰激凌。

从这个角度看，**冰激凌的销量**并不是由天气直接决定的，而是由人们**出门的愿望**和**吃冰激凌的愿望**共同决定的。

---

那么，我们是否可以这样做：

* 先用一个模型根据天气预测这些中间因素；
* 再用另一个模型根据这些中间因素预测冰激凌销量？

更进一步，我们是否可以把前一个模型的输出，直接作为后一个模型的输入，并将这两个模型连接成一个整体？

如果可以，那么在训练时，我们是不是可以把冰激凌销量的预测误差，通过反向传播传递回前面的模型，使两个模型能够同时被训练？

---

这种在模型内部自动学习中间表示的能力，正是深度神经网络的独特之处。

## 层

我们设想一下，是否可以把我们的模型分成两**层**（Layer）：

* 第一层：根据天气情况预测两个数据：人们出门的愿望，和吃冰激凌的愿望；
* 第二层：利用第一层的两个预测结果来预测冰激凌的销量。

这样的模型是否可行？能训练成功吗？


In [1]:
import numpy as np

np.random.seed(99)

## 数据集

### 训练数据：特征、标签

In [2]:
train_features = np.array([[22.5, 72.0],
                           [31.4, 45.0],
                           [19.8, 85.0],
                           [27.6, 63]])
train_labels = np.array([[95],
                        [210],
                        [70],
                        [155]])

### 测试数据：特征、标签

In [3]:
test_features = np.array([[28.1, 58.0]])
test_labels = np.array([[165]])

## 模型

两层的神经元网络模型，其数据流将变成传播链路：

1. **前向传播链路**：

```{figure} images/layer-forward.png
:align: center
:width: 480px
```

* $x$：特征值；
* $w_1, b_1$：第一层权重和偏置；
* $h$：中间值；
* $w_2, b_2$：第二层权重和偏置；
* $p$：预测值。

2. **梯度计算链路**：

```{figure} images/layer-gradient.png
:align: center
:width: 640px
```

* $p$：预测值；
* $y$：标签值；
* $l$：损失值；
* $d_2$：第二层误差项；
* $w_2$：第二层权重；
* $d_1$：第一层误差项。

3. **反向传播链路**

```{figure} images/layer-backward.png
:align: center
:width: 240px
```

* $d_2$：第二层误差项；
* $h$：中间值；
* $w_2, b_2$：第二层权重和偏置；
* $d_1$：第一层误差项；
* $x$：特征值；
* $w_1, b_1$：第一层权重和偏置。

---

两层模型训练的过程是多轮迭代，通过两层梯度，逐步调整两层参数，收敛到最优解。

根据微积分的导数链式法则，两层模型的反向传播链是可行的，而且这个过程不需要中间值的参与。

如果 $X$ 是天气预报，$W_1, b_1$ 是第一层的权重和偏置，那么人们出门和吃冰激凌的愿望 $H$ 就是：

$$
H = W_1 \cdot X + b_1
$$

$W_2, b_2$ 是第二层的权重和偏置，那么冰激凌的销量 $p$ 就是：

$$
p = W_2 \cdot H + b_2
$$

因此：

$$
p = W_2 \cdot (W_1 \cdot X + b_1) + b_2
$$

当我们开始计算两层的权重 $W$ 和偏置 $b$ 的梯度时，人们出门和吃冰激凌的愿望 $H$ 消失了。$H$ 虽然是前向传播过程中承上启下的重要一环，但是在梯度计算和反向传播的过程中却是不需要的。

### 参数：隐藏层权重、偏置

实践中，我们并不需要关心第一层模型预测出的中间值的实际含义。它们只是给第二层提供了训练数据。

因此第一层经常被称为**隐藏层**（Hidden Layer）。或者说，所有输出结果是中间值的层都是隐藏层。

---

权重是一个二维数组，数组的每一行等同于一个人工神经元的权重。利用 NumPy 提供的矢量计算能力，我们可以一次性完成所有中间结果的计算。

现在，我们让隐藏层预测 4 个中间值。我们还需要 4 个对应的偏置：

In [4]:
hidden_weight = np.random.rand(4, 2) / 2
hidden_bias = np.random.rand(4)

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 提示：</strong> 我们利用 NumPy 的随机数生成器，初始化了 4 组不同的权重和偏置。这样的机制将引导模型沿着不同方向推理中间值。
</div>

### 参数：输出层权重、偏置

第二层通常称为**输出层**（Output Layer）。或者说，最后一层是输出层。它的输出结果是我们真正需要的预测值。

输出层将有 4 个输入数据（隐藏层输出的 4 个中间值），并最终推理出一个预测值。我们也同样使用 NumPy 的随机数生成器来初始化权重和偏置：

In [ ]:
output_weight = np.random.rand(1, 4) / 4
output_bias = np.random.rand(1)

### 推理函数

In [6]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（均方误差）

In [7]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

In [8]:
def gradient(p, y):
    return - 2 * (y - p) / len(y)

### 反向函数

In [9]:
def backward(x, d, w, b, lr):
    w = w - d.T @ x * lr
    b = b - np.sum(d, axis=0) * lr
    return w, b

### 梯度反向函数

梯度反向函数用于从输出层误差项推导出隐藏层误差项。其本质就是输出层推理函数的导数。

In [10]:
def gradient_backward(d, w):
    return d @ w / len(d)

## 训练

### 超参数：学习率

In [11]:
LEARNING_RATE = 0.00001

### 超参数：批大小

In [12]:
BATCH_SIZE = 2

### 超参数：轮数

In [13]:
EPOCHS = 1000

### 迭代

两层的人工神经元网络模型，其前向传播、梯度计算、和反向传播的过程都将包括两层的运算。

In [14]:
for epoch in range(EPOCHS):
    for i in range(0, len(train_features), BATCH_SIZE):
        # 读入一个批次需要的多个训练数据：特征、标签
        features = train_features[i: i + BATCH_SIZE]
        labels = train_labels[i: i + BATCH_SIZE]

        # 推理中间数据
        hidden = forward(features, hidden_weight, hidden_bias)
        # 推理输出数据
        predictions = forward(hidden, output_weight, output_bias)

        # 计算输出层梯度
        output_delta = gradient(predictions, labels)
        # 计算隐藏层梯度
        hidden_delta = gradient_backward(output_delta, output_weight)

        # 用输出层梯度更新输出层参数
        output_weight, output_bias = backward(hidden, output_delta, output_weight, output_bias, LEARNING_RATE)
        # 用隐藏层梯度更新隐藏层参数
        hidden_weight, hidden_bias = backward(features, hidden_delta, hidden_weight, hidden_bias, LEARNING_RATE)

print(f"hidden weight: {hidden_weight}")
print(f"hidden bias: {hidden_bias}")
print(f"output weight: {output_weight}")
print(f"output bias: {output_bias}")

hidden weight: [[ 1.23922473 -0.1116876 ]
 [ 1.18929861 -0.32039931]
 [ 1.65942519 -0.16219998]
 [ 0.56455616 -0.1290233 ]]
hidden bias: [1.00609913 0.01971251 0.79170399 0.75393082]
output weight: [[1.65622912 1.61380819 2.24198115 0.79391568]]
output bias: [1.01922324]


## 验证

### 推理

模型推理的过程也要经过两层的推理函数。

In [15]:
hidden = forward(test_features, hidden_weight, hidden_bias)
predictions = forward(hidden, output_weight, output_bias)
print(f'predictions: {predictions}')

predictions: [[166.08341955]]


### 评估

In [16]:
error = mse_loss(predictions, test_labels)
print(f'loss: {error}')

loss: 1.173797931826622


实践证明，两层网络模型是可行的，但是也没有比单层网络模型表现得好太多。